In [1]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/fastspeech2-en-ljspeech",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)


2022-02-10 13:09:00 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-02-10 13:09:03 | INFO | fairseq.tasks.speech_to_text | dictionary size (vocab.txt): 75


In [2]:
model = models
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator(model, cfg)

text = "Hello, this is a test run."

sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)

2022-02-10 13:09:05 | INFO | fairseq.models.text_to_speech.vocoder | loaded HiFiGAN checkpoint from C:/Users/runze/.cache/fairseq/facebook--fastspeech2-en-ljspeech.main.a3e3e5e2e62bb7ca7514b11aa469e9c5b01a20bf/hifigan.bin
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\runze\AppData\Roaming\nltk_data...
